In [15]:
import math
import graphviz
import math
import copy
import sys
import arff

------
### Algorytm TABU
<img src="../img/tabu.png" style="height: 350px">

In [4]:
class TabuSearch(object):
    def __init__(self, score_method, number_of_parents, number_of_iterations, tabu_length, attributes=None, sample_data=None):
        self.attributes = attributes
        self.sample_data = sample_data
        self.score_method = score_method
        self.number_of_parents = number_of_parents
        self.number_of_iterations = number_of_iterations
        self.tabu_length = tabu_length
        self.graph_checker = GraphChecker()
        
    def find_optimal_net(self):
        tabu_list = []
        network = self.prepare_initial_network()
        best_score = self.compute_metric(network)
        
        for progress_iterator in log_progress(range(0, self.number_of_iterations), name="Iteracja", every=1):
            best_candidate, candidate_score, tabu_list, has_no_more_good_solutions = self.find_best_operation(network, tabu_list)
            if has_no_more_good_solutions:
                break;
            
            if candidate_score > best_score:
                best_score = candidate_score
                network = best_candidate
            
            self.check_tabu_list_length(tabu_list)
        return network
    
    def find_best_operation(self, network, tabu_list):
        add_child, add_parent, add_candidate, add_best_score = self.find_best_addition(network, tabu_list)
        del_child, del_parent, del_candidate, del_best_score = self.find_best_deletion(network, tabu_list)
        
        best_score = add_best_score if add_best_score > del_best_score else del_best_score
        best_action = "+" if add_best_score > del_best_score else "-"
        best_parent = add_parent if add_best_score > del_best_score else del_parent
        best_child = add_child if add_best_score > del_best_score else del_child
        best_net = add_candidate if add_best_score > del_best_score else del_candidate
        
        has_no_more_good_solutions = (best_child == '' and best_parent == '') #in case ADD and DEL gives int.min

        if not has_no_more_good_solutions:
            tabu_list.append({"child": best_child, "parent": best_parent, "action": best_action})
        
        return (best_net, best_score, tabu_list, has_no_more_good_solutions)
        
            
    def find_best_addition(self, network, tabu):
        temp_network = copy.deepcopy(network)
        best_network = copy.deepcopy(network)
        best_parent = ""
        best_child = ""
        best_score = -sys.maxsize -1
        
        #finding best addition: first_node (parent) → second_node (child)
        for first_node in temp_network:
            for second_node in temp_network:
                if (first_node['name'] != second_node['name'] 
                    and not self.check_if_node_already_has_parent(second_node['name'], first_node['name'], best_network) 
                    and not self.has_max_parents(second_node['name'], best_network)
                    and not self.is_in_tabu(second_node['name'], first_node['name'], "+", tabu)):
                    
                    candidate_network = self.add_parent_to_node(second_node['name'], first_node, copy.deepcopy(temp_network))
                    
                    if self.graph_checker.is_graph_acyclic(candidate_network):
                        continue
                    
                    score = self.compute_metric(candidate_network)
                    
                    if (score > best_score):
                        best_score = score
                        best_parent = first_node['name']
                        best_child = second_node['name']
                        best_network = candidate_network

        return (best_child, best_parent, best_network, best_score)
    
    def find_best_deletion(self, network, tabu):
        temp_network = copy.deepcopy(network)
        best_network = copy.deepcopy(network)
        best_parent = ""
        best_child = ""
        best_score = -sys.maxsize -1
        
        #finding best deletion: parent (old parent) ↛ node (old child)
        for node in temp_network:
            for parent in node['parents']:
                if not self.is_in_tabu(node['name'], parent['name'], "-", tabu):
                    candidate_network = self.delete_parent_from_node(node['name'], parent['name'], temp_network)
                    candidate_score = self.compute_metric(candidate_network)
                    
                    if (candidate_score > best_score):
                        best_score = candidate_score
                        best_parent = parent['name']
                        best_child = node['name']
                        best_network = candidate_network
                        
        return (best_child, best_parent, best_network, best_score)
    
    def is_in_tabu(self, child, parent, action, tabu):
        for entry in tabu:
            if (entry['child'] == child 
                and entry['parent'] == parent 
                and entry['action'] == action):
                return True
        return False
    
    def prepare_initial_network(self):
        initial_bayes_network = []
        for attribute in self.attributes:
            initial_bayes_network.append({'r': attribute['states'], 'name': attribute['name'], 'parents': []})
        return initial_bayes_network
    
    def check_tabu_list_length(self, tabu_list):
        if len(tabu_list) > self.tabu_length:
            del tabu_list[0]
            
    def check_if_node_already_has_parent(self, child_name, parent_name, net):
        for node in net:
            if node['name'] == child_name:
                for parent in node['parents']:
                    if parent['name'] == parent_name:
                        return True
        return False

    def has_max_parents(self, node_name, net):
        for node in net:
            if node['name'] == node_name:
                if len(node['parents']) >= self.number_of_parents:
                    return True
                else:
                    return False
        raise ValueError(self.node_name + " is not found in given net")
        
    def add_parent_to_node(self, node_name, parent, net):
        for node in net:
            if node['name'] == node_name:
                node['parents'].append({'name': parent['name'], 'q': parent['r']})
        return net
    
    def delete_parent_from_node(self, node_name, parent_name, net):
        temp_net = copy.deepcopy(net)
        for node in temp_net:
            if node['name'] == node_name:
                for parent_index, parent in enumerate(node['parents']):
                    if parent_name == parent['name']:
                        del node['parents'][parent_index]
                        return temp_net
        
    def compute_metric(self, net):
        if self.score_method == 'aic':
            return AICMetric().compute_aic_metric(net, self.sample_data)
        elif self.score_method == 'mdl':
            return MDLMetric().compute_mdl_metric(net, self.sample_data)
        elif self.score_method == 'bayes':
            return BayesianMetric(net, self.sample_data).compute_bayesian_metric()
        elif self.score_method == 'entropy':
            return EntropyMetric(net, self.sample_data).compute_entropy_metric()
        
        raise ValueError(self.score_method + " is not a valid score method!")
        
    def prepare_graph(self, net):
        graph = []
        
        for node in net:
            graph.append({"node": node['name'], "children": []})
            
        for node in net:
            for parent in node['parents']:
                for graph_node in graph:
                    if graph_node['node'] == parent['name']:
                        graph_node['children'].append(node['name'])
                        
        return graph